In [ ]:
import pandas as pd
import re
from googletrans import Translator

# Load the dataset
file_path = 'train.csv'
dataset = pd.read_csv(file_path)

# Initialize the translator
translator = Translator()

# Function to clean and translate text
def translate_text(text):
    # Clean the text by removing unwanted characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    try:
        # Translate from Hinglish to English
        translated = translator.translate(text, src='hi', dest='en')
        return translated.text
    except Exception as e:
        return f"Translation error: {e}"

# Apply the translation function to the 'crimeaditionalinfo' column
dataset['translated_text'] = dataset['crimeaditionalinfo'].apply(translate_text)

# Display the original and translated texts
dataset[['crimeaditionalinfo', 'translated_text']].head()


In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

model_name = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)

def translate_m2m100(text):
    tokenizer.src_lang = "hi"  # You may need to set this to match Hinglish or Hindi-English
    encoded = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(**encoded)
    translated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    return translated_text

print(translate_m2m100("RUPAY KA RECHAREGE PHONE PE SE KARNE..."))

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# Load the tokenizer and model for Hindi to English translation
model_name = 'Helsinki-NLP/opus-mt-hi-en'  # Model trained for Hindi to English
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Function to translate text from Hinglish to English
def translate_text(text):
    # Tokenize the input text and get model predictions
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    translated_tokens = model.generate(**inputs)
    # Decode the translated tokens to get the final output
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_text

# Example usage with a Hinglish sentence
hinglish_sentence = "main college jaa raha hoon. it is going to be fun"
english_translation = translate_text(hinglish_sentence)
print("Translated Text:", english_translation)


In [5]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

article_hi = "संयुक्त राष्ट्र के प्रमुख का कहना है कि सीरिया में कोई सैन्य समाधान नहीं है"
article_ar = "الأمين العام للأمم المتحدة يقول إنه لا يوجد حل عسكري في سوريا."

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# translate Hindi to French
tokenizer.src_lang = "hi_IN"
encoded_hi = tokenizer(article_hi, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=tokenizer.lang_code_to_id["fr_XX"]
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "Le chef de l 'ONU affirme qu 'il n 'y a pas de solution militaire dans la Syrie."

# translate Arabic to English
tokenizer.src_lang = "ar_AR"
encoded_ar = tokenizer(article_ar, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_ar,
    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "The Secretary-General of the United Nations says there is no military solution in Syria."


model.safetensors:  33%|###2      | 797M/2.44G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/facebook/mbart-large-50-many-to-many-mmt/10ef2c7b94d92f301d64ed29825ed05cb5374f347d8db7755de5474295e07ff7?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1731748956&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMTc0ODk1Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9mYWNlYm9vay9tYmFydC1sYXJnZS01MC1tYW55LXRvLW1hbnktbW10LzEwZWYyYzdiOTRkOTJmMzAxZDY0ZWQyOTgyNWVkMDVjYjUzNzRmMzQ3ZDhkYjc3NTVkZTU0NzQyOTVlMDdmZjc%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=SqYE6XjM07RGZcUTDVCvmIyxBS%7ErOCfbDvTiKlP9iIScJXKfd7E%7Ewpp1%7EtDET1WfNaePoBkWpYuvAPL9EFVzDPcel4ziO7G5bAZa9yoYvMRUxjn0JGOgZRmCfOCTIxzIwsWARe7vWyOiFlgJJgimTKCYmSG546LRmWDao94rL60YsGfYkDR4Ur6VkrqogXjkFt72d1CX9K2by0uK89XJa-BaO83VJsgwXKK7h2m-Mqnbx%7ETcZJlvuXRtDzWxYr1z%7EOeS%7EaUT-yfb4OgpWC8gm6VqQq-3l%7EshZBsUlHK6Rpjsm2FaaTorWuaLNeVsaSVUIJsq3Fz%7E54xAIfq

model.safetensors:  33%|###2      | 797M/2.44G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/facebook/mbart-large-50-many-to-many-mmt/10ef2c7b94d92f301d64ed29825ed05cb5374f347d8db7755de5474295e07ff7?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1731748956&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMTc0ODk1Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9mYWNlYm9vay9tYmFydC1sYXJnZS01MC1tYW55LXRvLW1hbnktbW10LzEwZWYyYzdiOTRkOTJmMzAxZDY0ZWQyOTgyNWVkMDVjYjUzNzRmMzQ3ZDhkYjc3NTVkZTU0NzQyOTVlMDdmZjc%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=SqYE6XjM07RGZcUTDVCvmIyxBS%7ErOCfbDvTiKlP9iIScJXKfd7E%7Ewpp1%7EtDET1WfNaePoBkWpYuvAPL9EFVzDPcel4ziO7G5bAZa9yoYvMRUxjn0JGOgZRmCfOCTIxzIwsWARe7vWyOiFlgJJgimTKCYmSG546LRmWDao94rL60YsGfYkDR4Ur6VkrqogXjkFt72d1CX9K2by0uK89XJa-BaO83VJsgwXKK7h2m-Mqnbx%7ETcZJlvuXRtDzWxYr1z%7EOeS%7EaUT-yfb4OgpWC8gm6VqQq-3l%7EshZBsUlHK6Rpjsm2FaaTorWuaLNeVsaSVUIJsq3Fz%7E54xAIfq

model.safetensors:  33%|###2      | 797M/2.44G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/facebook/mbart-large-50-many-to-many-mmt/10ef2c7b94d92f301d64ed29825ed05cb5374f347d8db7755de5474295e07ff7?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1731748956&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMTc0ODk1Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9mYWNlYm9vay9tYmFydC1sYXJnZS01MC1tYW55LXRvLW1hbnktbW10LzEwZWYyYzdiOTRkOTJmMzAxZDY0ZWQyOTgyNWVkMDVjYjUzNzRmMzQ3ZDhkYjc3NTVkZTU0NzQyOTVlMDdmZjc%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=SqYE6XjM07RGZcUTDVCvmIyxBS%7ErOCfbDvTiKlP9iIScJXKfd7E%7Ewpp1%7EtDET1WfNaePoBkWpYuvAPL9EFVzDPcel4ziO7G5bAZa9yoYvMRUxjn0JGOgZRmCfOCTIxzIwsWARe7vWyOiFlgJJgimTKCYmSG546LRmWDao94rL60YsGfYkDR4Ur6VkrqogXjkFt72d1CX9K2by0uK89XJa-BaO83VJsgwXKK7h2m-Mqnbx%7ETcZJlvuXRtDzWxYr1z%7EOeS%7EaUT-yfb4OgpWC8gm6VqQq-3l%7EshZBsUlHK6Rpjsm2FaaTorWuaLNeVsaSVUIJsq3Fz%7E54xAIfq

model.safetensors:  33%|###3      | 807M/2.44G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/facebook/mbart-large-50-many-to-many-mmt/10ef2c7b94d92f301d64ed29825ed05cb5374f347d8db7755de5474295e07ff7?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1731748956&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMTc0ODk1Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9mYWNlYm9vay9tYmFydC1sYXJnZS01MC1tYW55LXRvLW1hbnktbW10LzEwZWYyYzdiOTRkOTJmMzAxZDY0ZWQyOTgyNWVkMDVjYjUzNzRmMzQ3ZDhkYjc3NTVkZTU0NzQyOTVlMDdmZjc%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=SqYE6XjM07RGZcUTDVCvmIyxBS%7ErOCfbDvTiKlP9iIScJXKfd7E%7Ewpp1%7EtDET1WfNaePoBkWpYuvAPL9EFVzDPcel4ziO7G5bAZa9yoYvMRUxjn0JGOgZRmCfOCTIxzIwsWARe7vWyOiFlgJJgimTKCYmSG546LRmWDao94rL60YsGfYkDR4Ur6VkrqogXjkFt72d1CX9K2by0uK89XJa-BaO83VJsgwXKK7h2m-Mqnbx%7ETcZJlvuXRtDzWxYr1z%7EOeS%7EaUT-yfb4OgpWC8gm6VqQq-3l%7EshZBsUlHK6Rpjsm2FaaTorWuaLNeVsaSVUIJsq3Fz%7E54xAIfq

SSLError: (MaxRetryError("HTTPSConnectionPool(host='cdn-lfs.hf.co', port=443): Max retries exceeded with url: /facebook/mbart-large-50-many-to-many-mmt/10ef2c7b94d92f301d64ed29825ed05cb5374f347d8db7755de5474295e07ff7?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1731748956&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMTc0ODk1Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9mYWNlYm9vay9tYmFydC1sYXJnZS01MC1tYW55LXRvLW1hbnktbW10LzEwZWYyYzdiOTRkOTJmMzAxZDY0ZWQyOTgyNWVkMDVjYjUzNzRmMzQ3ZDhkYjc3NTVkZTU0NzQyOTVlMDdmZjc~cmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=SqYE6XjM07RGZcUTDVCvmIyxBS~rOCfbDvTiKlP9iIScJXKfd7E~wpp1~tDET1WfNaePoBkWpYuvAPL9EFVzDPcel4ziO7G5bAZa9yoYvMRUxjn0JGOgZRmCfOCTIxzIwsWARe7vWyOiFlgJJgimTKCYmSG546LRmWDao94rL60YsGfYkDR4Ur6VkrqogXjkFt72d1CX9K2by0uK89XJa-BaO83VJsgwXKK7h2m-Mqnbx~TcZJlvuXRtDzWxYr1z~OeS~aUT-yfb4OgpWC8gm6VqQq-3l~shZBsUl